# Combining Figures

xarray-plotly provides helper functions to combine multiple figures:

- **`overlay_figures`**: Overlay traces on the same axes
- **`add_secondary_y`**: Plot with two independent y-axes

In [ ]:
import numpy as np
import xarray as xr

from xarray_plotly import add_secondary_y, config, overlay_figures, xpx

config.notebook()

## Sample Data

In [ ]:
# Time series with categories
np.random.seed(42)
time = np.arange(50)

sales = xr.DataArray(
    np.cumsum(np.random.randn(50, 3), axis=0) + 100,
    dims=["day", "product"],
    coords={"day": time, "product": ["Widget", "Gadget", "Gizmo"]},
    name="Sales",
)

# Two variables with different scales
temperature = xr.DataArray(
    20 + 10 * np.sin(time / 10) + np.random.randn(50),
    dims=["day"],
    coords={"day": time},
    name="Temperature",
    attrs={"units": "°C"},
)

precipitation = xr.DataArray(
    np.maximum(0, 5 + 10 * np.random.randn(50)),
    dims=["day"],
    coords={"day": time},
    name="Precipitation",
    attrs={"units": "mm"},
)

## overlay_figures

Overlay multiple figures on the same axes. Useful for combining different plot types.

In [ ]:
# Area chart with line overlay
area_fig = xpx(sales).area()
line_fig = xpx(sales).line()

# Update line style to make it visible on top of area
line_fig.update_traces(line={"color": "black", "width": 1})

combined = overlay_figures(area_fig, line_fig)
combined.update_layout(title="Sales: Area with Line Overlay")
combined

### Multiple Overlays

You can overlay more than two figures.

In [ ]:
# Three different visualizations of the same data
area = xpx(sales).area()
line = xpx(sales).line()
scatter = xpx(sales).scatter()

# Style them differently
line.update_traces(line={"color": "black", "width": 1, "dash": "dot"})
scatter.update_traces(marker={"color": "black", "size": 4})

combined = overlay_figures(area, line, scatter)
combined.update_layout(title="Sales: Area + Line + Scatter")
combined

### With Facets

`overlay_figures` works with faceted figures as long as both have the same structure.

In [ ]:
area_faceted = xpx(sales).area(facet_col="product")
line_faceted = xpx(sales).line(facet_col="product")
line_faceted.update_traces(line={"color": "black", "width": 2})

combined = overlay_figures(area_faceted, line_faceted)
combined.update_layout(title="Faceted: Area + Line per Product")
combined

### With Animation

Animation frames are merged correctly.

In [ ]:
# Create animated data
animated_data = xr.DataArray(
    np.random.rand(20, 3, 5).cumsum(axis=0),
    dims=["x", "category", "frame"],
    coords={
        "x": np.arange(20),
        "category": ["A", "B", "C"],
        "frame": [1, 2, 3, 4, 5],
    },
    name="Value",
)

area_anim = xpx(animated_data).area(animation_frame="frame")
line_anim = xpx(animated_data).line(animation_frame="frame")
line_anim.update_traces(line={"color": "black", "width": 2})

combined = overlay_figures(area_anim, line_anim)
combined.update_layout(title="Animated: Area + Line")
combined

## add_secondary_y

Plot two variables with different scales using independent y-axes.

In [ ]:
# Temperature on left y-axis, precipitation on right y-axis
temp_fig = xpx(temperature).line()
temp_fig.update_traces(line={"color": "red"})

precip_fig = xpx(precipitation).bar()
precip_fig.update_traces(marker={"color": "blue", "opacity": 0.6})

combined = add_secondary_y(temp_fig, precip_fig)
combined.update_layout(title="Weather: Temperature & Precipitation")
combined

### Custom Y-Axis Title

Use `secondary_y_title` to customize the right y-axis label.

In [ ]:
temp_fig = xpx(temperature).line()
temp_fig.update_traces(line={"color": "red", "width": 2})

precip_fig = xpx(precipitation).bar()
precip_fig.update_traces(marker={"color": "steelblue"})

combined = add_secondary_y(
    temp_fig,
    precip_fig,
    secondary_y_title="Rainfall (mm)",
)
combined.update_layout(
    title="Weather Data",
    yaxis_title="Temperature (°C)",
    legend={"orientation": "h", "y": 1.1},
)
combined

### With Animation

`add_secondary_y` also supports animated figures.

In [ ]:
# Create two animated variables with different scales
var1 = xr.DataArray(
    np.random.rand(20, 5) * 100,
    dims=["x", "frame"],
    coords={"x": np.arange(20), "frame": [1, 2, 3, 4, 5]},
    name="Metric A",
)

var2 = xr.DataArray(
    np.random.rand(20, 5) * 10,
    dims=["x", "frame"],
    coords={"x": np.arange(20), "frame": [1, 2, 3, 4, 5]},
    name="Metric B",
)

fig1 = xpx(var1).line(animation_frame="frame")
fig2 = xpx(var2).bar(animation_frame="frame")
fig2.update_traces(marker={"opacity": 0.5})

combined = add_secondary_y(fig1, fig2)
combined.update_layout(title="Animated Dual Y-Axis")
combined

## Limitations

### overlay_figures
- Overlay must have same or fewer subplots than base
- Animation frames must match (or overlay must be static)

### add_secondary_y
- Does not support faceted figures (subplots)
- Animation frames must match (or secondary must be static)